In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv',
    parse_dates=['date']
)

item_names = sorted(df['type_name'].unique())
item_dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Type or select an item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)
donchian_slider = widgets.IntSlider(
    value=20, min=5, max=60, step=1, description='Donchian Window', continuous_update=False
)
output = widgets.Output()
display(item_dropdown, donchian_slider, output)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=(' Tyrant Blue Sa…

IntSlider(value=20, continuous_update=False, description='Donchian Window', max=60, min=5)

Output()

In [2]:
def plot_donchian_breakouts(item, donchian_window):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()

        # Calculate Donchian upper/lower bands
        item_df['donchian_high'] = item_df['highest'].rolling(window=donchian_window, min_periods=1).max()
        item_df['donchian_low'] = item_df['lowest'].rolling(window=donchian_window, min_periods=1).min()

        # Find breakout days
        breakout_up = (item_df['average'] > item_df['donchian_high'].shift(1))
        breakout_down = (item_df['average'] < item_df['donchian_low'].shift(1))

        print(f"Donchian window: {donchian_window} days | Up breakouts: {breakout_up.sum()} | Down breakouts: {breakout_down.sum()}")

        plt.figure(figsize=(12,6))
        plt.plot(item_df['date'], item_df['average'], label='Price', color='blue')
        plt.plot(item_df['date'], item_df['donchian_high'], label=f'Donchian High ({donchian_window})', color='deepskyblue', linestyle='--')
        plt.plot(item_df['date'], item_df['donchian_low'], label=f'Donchian Low ({donchian_window})', color='purple', linestyle='--')
        plt.fill_between(item_df['date'], item_df['donchian_low'], item_df['donchian_high'], color='cyan', alpha=0.12)
        plt.scatter(item_df.loc[breakout_up, 'date'], item_df.loc[breakout_up, 'average'],
                    label='Up Breakout', color='lime', edgecolor='k', marker='o', s=90, zorder=5)
        plt.scatter(item_df.loc[breakout_down, 'date'], item_df.loc[breakout_down, 'average'],
                    label='Down Breakout', color='orange', edgecolor='k', marker='o', s=90, zorder=5)
        plt.title(f"{item} — Donchian Channel Breakouts\n{donchian_window}-Day Window")
        plt.xlabel("Date")
        plt.ylabel("Average Price (ISK)")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

widgets.interactive_output(
    plot_donchian_breakouts,
    {'item': item_dropdown, 'donchian_window': donchian_slider}
)


Output()